# Challenge Data Science - Semana 1

Para o challenge, a Alura Voz entrou em contato e pediu que fosse feito um estudo para verificar, entender e reduzir a taxa de evasão dos clientes, o **Churn Rate**.

Como primeiro passo nesse estudo, será feita a identificação de clientes que tem a maior chance de deixar a empresa, com base em algumas características dos clientes ou dos planos contratados por eles.

A empresa disponibilizou então uma base de dados de seus clientes, constando informações pessoais, financeiras e de contrato dos clientes. Com base nesses dados, é esperado que seja possível classificar os clientes, encontrando assim potenciais candidatos a deixar a empresa.

A base de dados foi disponibilizada no formato JSON (**J**ava**S**cript **O**bject **N**otation).

Primeiramente, os dados devem ser analisados e tratados. Podem haver inconsistências e campos sem dados, o que pode levar a conclusões precipitadas ou equivocadas.

## Importando os Dados

In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/vinicius-pf/Challenge_DataScience/main/Semana%201/dados/dados.json'


dados = pd.read_json(
    path_or_buf = url,
    orient = 'columns'
)

In [ ]:
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Os dados não estão normalizados, ou seja: as células contém informações que também podem ser colunas do dataset. Antes de prosseguir com as análises, é necessário normalizar os dados. Com isso, cada coluna terá uma informação única.

### Normalizando os dados

Primeiramente, cada coluna com informações relevantes será transformada em uma tabela.

In [ ]:
dados_customer = pd.json_normalize(data = dados.customer)
dados_customer.head()

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4
3,Male,1,Yes,No,13
4,Female,1,Yes,No,3


In [ ]:
dados_phone = pd.json_normalize(data = dados.phone)
dados_phone.head()

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes
2,Yes,No
3,Yes,No
4,Yes,No


In [ ]:
dados_internet = pd.json_normalize(data = dados.internet)
dados_internet.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes
2,Fiber optic,No,No,Yes,No,No,No
3,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,Fiber optic,No,No,No,Yes,Yes,No


In [ ]:
dados_account = pd.json_normalize(data = dados.account)
dados_account.head()

,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,One year,Yes,Mailed check,65.6,593.3
1,Month-to-month,No,Mailed check,59.9,542.4
2,Month-to-month,Yes,Electronic check,73.9,280.85
3,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Month-to-month,Yes,Mailed check,83.9,267.4


Com os DataFrame criados, as informações devem ser unificadas em um único DataFrame.

É possível manter os dados separados, porém a análise dos mesmos ficará mais longa. A opção de manter os dados separados é eficaz para a criação de banco de dados *Star Schema*.

Para a união dos DataFrame, irei uni-los em pares. Isso permite que eu verifique meu progresso e garanta que não está ocorrendo nenhum erro.

Primeiramente irei unir as informações contidas em *customer* e *phone*. Para garantir que todas as informações foram mantidas, irei verificar o tamanho final dos DataFrames.

In [ ]:
dados_customer_phone = pd.merge(
    left = dados_customer,
    right = dados_phone,
    left_index = True,
    right_index = True
)

In [ ]:
print(f'O primeiro DataFrame contém {dados_customer.shape[0]} linhas. O segundo contém {dados_phone.shape[0]}. A união contém {dados_customer_phone.shape[0]}')

O primeiro DataFrame contém 7267 linhas. O segundo contém 7267. A união contém 7267


In [ ]:
dados_customer_phone.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines
0,Female,0,Yes,Yes,9,Yes,No
1,Male,0,No,No,9,Yes,Yes
2,Male,0,No,No,4,Yes,No
3,Male,1,Yes,No,13,Yes,No
4,Female,1,Yes,No,3,Yes,No


Conferindo as informações com o arquivo original, posso continuar e fazer a união das informações referentes as colunas *internet* e *account*

In [ ]:
dados_internet_account = pd.merge(
    left = dados_internet,
    right = dados_account,
    left_index = True,
    right_index = True
)

In [ ]:
print(f'O primeiro DataFrame contém {dados_internet.shape[0]} linhas. O segundo contém {dados_account.shape[0]}. A união contém {dados_internet_account.shape[0]}')

O primeiro DataFrame contém 7267 linhas. O segundo contém 7267. A união contém 7267


In [ ]:
dados_internet_account.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Após isso, podemos unir os dois DataFrames, e por último, incluir as informações em um novo DataFrames.

In [ ]:
dados_colunas = pd.merge(
    left = dados_customer_phone,
    right = dados_internet_account,
    left_index = True,
    right_index = True
)

In [ ]:
dados_normal = pd.merge(
    left = dados,
    right = dados_colunas,
    left_index = True,
    right_index = True
)

In [ ]:
dados_normal.head()

,customerID,Churn,customer,phone,internet,account,gender,SeniorCitizen,Partner,Dependents,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '...",Female,0,Yes,Yes,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli...",Male,0,No,No,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli...",Male,0,No,No,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli...",Male,1,Yes,No,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli...",Female,1,Yes,No,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Além da criação do novo DataFrame, podemos excluir as colunas que foram normalizadas.

In [ ]:
colunas = ['internet', 'account', 'phone', 'customer']

dados_normal.drop(labels = colunas ,axis = 1, inplace = True)
dados_normal.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## Entendendo os dados

Após a normalização dos dados, será possível fazer uma análise exploratória dos dados para entender as informações contidas em cada coluna do mesmo.

In [ ]:
dados_normal.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [ ]:
print(f'A base de dados contém {dados_normal.shape[0]} registros e {dados_normal.shape[1]} colunas')

A base de dados contém 7267 registros e 21 colunas


In [ ]:
dados_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


São 21 colunas. Em uma primeira análise, não há células com valor faltante na base de dados. 

Além dessa análise, a empresa também disponibilizou um dicionário com uma descrição de cada coluna.

Nome da coluna | Dicionário da empresa
-------|------------------
customerID| Código único para o cliente da empresa
Churn| se o cliente deixou ou não a empresa
gender| gênero (masculino e feminino)
SeniorCitizen| informação sobre um cliente ter ou não idade igual ou maior que 65 anos
Partner| se o cliente possui ou não um parceiro ou parceira
Dependents| se o cliente possui ou não dependentes
tenure| meses de contrato do cliente
PhoneService| assinatura de serviço telefônico
MultipleLines| assinatura de mais de uma linha de telefone
InternetService| assinatura de um provedor internet
OnlineSecurity| assinatura adicional de segurança online
OnlineBackup| assinatura adicional de backup online
DeviceProtection| assinatura adicional de proteção no dispositivo
TechSupport| assinatura adicional de suporte técnico, menos tempo de espera
StreamingTV| assinatura de TV a cabo
StreamingMovies| assinatura de streaming de filmes
Contract| tipo de contrato
PaperlessBilling| se o cliente prefere receber online a fatura
PaymentMethod| forma de pagamento
Charges.Monthly| total de todos os serviços do cliente por mês
Charges.Total| total gasto pelo cliente

Esse dicionário ajuda a entender um pouco mais o significado das colunas.

Além disso, é possível perceber algumas inconsistências nos dados, como falta de padronização em algumas informações e colunas tendo o tipo equivocado dos dados.

Para encontrar mais discrepâncias e possíveis erros, deverei analisar mais profundamente cada coluna.

## Analisando as colunas

Como algumas etapas serão as mesmas para todas as colunas, irei definir uma função que irá trazer as informações importantes para todas as colunas. Além das análises definidas na função, será possível também criação de gráficos para análise visual dos dados.

Após as análises, será possível encontrar as principais discrepâncias na base de dados, além de encontrar dados faltantes e duplicados. 

#### Definindo a função

In [ ]:
def analisa_coluna(coluna):

    unicos = dados_normal[coluna].unique()
    contagem = dados_normal[coluna].value_counts()
    tipo = dados_normal[coluna].dtype

    print(f'Valores únicos:')
    print(unicos)
    print('----------------')
    print(f'Contagem de valores:')
    print(contagem)
    print('----------------')
    print(f'Tipo da coluna:')
    print(tipo)
    print('----------------')


#### Coluna 'customerID'

In [ ]:
analisa_coluna('customerID')

Valores únicos:
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
----------------
Contagem de valores:
0002-ORFBO    1
6614-VBEGU    1
6637-KYRCV    1
6635-MYYYZ    1
6635-CPNUN    1
             ..
3374-TTZTK    1
3374-PZLXD    1
3374-LXDEV    1
3373-YZZYM    1
9995-HOTOH    1
Name: customerID, Length: 7267, dtype: int64
----------------
Tipo da coluna:
object
----------------


A primeira coluna parece estar de acordo com o dicionário da empresa. Não possue valores em brancos e apenas possue valores únicos. 

Caso houvessem valores em branco, as linhas correspondentes seriam descartadas. Caso houvessem valores duplicados, eles seriam analizados individualmente para saber o motivo.

#### Coluna 'Churn'

In [ ]:
analisa_coluna('Churn')

Valores únicos:
['No' 'Yes' '']
----------------
Contagem de valores:
No     5174
Yes    1869
        224
Name: Churn, dtype: int64
----------------
Tipo da coluna:
object
----------------


A segunda coluna apresenta valores que devem ser tratados. Apesar da primeira análise não encontrar valores em branco, há campos vazios. Esses campos serão analisados com mais clareza para tomar uma decisão mais correta no momento do tratamento dos dados.

#### Coluna 'gender'

In [ ]:
analisa_coluna('gender')

Valores únicos:
['Female' 'Male']
----------------
Contagem de valores:
Male      3675
Female    3592
Name: gender, dtype: int64
----------------
Tipo da coluna:
object
----------------


A coluna de gêneros também está com os dados condizentes com o dicionário apresentado.

Há uma diferença pequena de valores entre *male* e *female*.

#### Coluna 'SeniorCitizen'

In [ ]:
analisa_coluna('SeniorCitizen')

Valores únicos:
[0 1]
----------------
Contagem de valores:
0    6085
1    1182
Name: SeniorCitizen, dtype: int64
----------------
Tipo da coluna:
int64
----------------


Nesta coluna, encontrei a primeira coluna com valores booleanos. Pela contagem de valores, o valor 0 significa que a idade da pessoa não é maior do que 65 anos, e 1 quando for.

A coluna está com o tipo errado, por conta do padrão utilizado. 

Junto com as informações já vistas, ela parece ser a única que segue esse padrão, sendo que as outras colunas que apresentam valores booleanos parecem ter valores 'Yes' e 'No'.

#### Coluna 'Partner'

In [ ]:
analisa_coluna('Partner')

Valores únicos:
['Yes' 'No']
----------------
Contagem de valores:
No     3749
Yes    3518
Name: Partner, dtype: int64
----------------
Tipo da coluna:
object
----------------


A coluna 'Partner', de acordo com o dicionário, informa se o cliente possue ou não um parceiro ou uma parceira. Novamente é uma coluna com valores booleanos. Ao contrário da coluna anterior, os valores utilizados são 'Yes' e 'No'. 

#### Coluna 'Dependents'

In [ ]:
analisa_coluna('Dependents')

Valores únicos:
['Yes' 'No']
----------------
Contagem de valores:
No     5086
Yes    2181
Name: Dependents, dtype: int64
----------------
Tipo da coluna:
object
----------------


A coluna traz as informações de se o cliente possue ou não dependentes. Ao utilizar a mesma notação que a coluna `Partner`, se percebe que a coluna `SeniorCitizen` deverá ser corrigida. 

Apesar de trazer a informação de dependentes, não consta, na base de dados, o número dos dependentes de cada pessoa.

#### Coluna 'tenure'

In [ ]:
analisa_coluna('tenure')

Valores únicos:
[ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 11 37 49 66
 67 20 43 59 12 27  2 25 29 14 35 64 39 40  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 51 31 36 17 18 38 42
  0]
----------------
Contagem de valores:
1     634
72    369
2     246
3     207
4     185
     ... 
38     60
39     59
44     54
36     50
0      11
Name: tenure, Length: 73, dtype: int64
----------------
Tipo da coluna:
int64
----------------


A coluna traz a quantidade de meses de contrato do cliente. Não há valores em branco.

Apesar de contar com valores numéricos, a variável relativa a coluna parece ser uma variável categórica. Será mantido o tipo da coluna, para que seja possível fazer análises quanto ao tempo de contrato de forma correta e direta.

Nota-se que há clientes que ainda não concluiram 1 mês de contrato e o maior número de clientes está com 1 mês de contrato. Essa coluna pode explicar o motivo de existirem dados em branco na coluna `Churn`. Isso será analizado no futuro.

#### Coluna 'PhoneService'

In [ ]:
analisa_coluna('PhoneService')

Valores únicos:
['Yes' 'No']
----------------
Contagem de valores:
Yes    6560
No      707
Name: PhoneService, dtype: int64
----------------
Tipo da coluna:
object
----------------


Dos 7267 clientes, 707 não possuem serviço de telefone. Além disso, percebe-se a padronização em valores 'Yes' e 'No' para colunas com informações booleanas.

#### Coluna 'MultipleLines'

In [ ]:
analisa_coluna('MultipleLines')

Valores únicos:
['No' 'Yes' 'No phone service']
----------------
Contagem de valores:
No                  3495
Yes                 3065
No phone service     707
Name: MultipleLines, dtype: int64
----------------
Tipo da coluna:
object
----------------


Nessa coluna, além dos valores de 'Yes' e 'No', também há um terceiro valor, 'No phone service'. Esse terceiro valor corresponde aos clientes que não possuem serviço de telefone contratado com a empresa. 



#### Coluna 'InternetService'

In [ ]:
analisa_coluna('InternetService')

Valores únicos:
['DSL' 'Fiber optic' 'No']
----------------
Contagem de valores:
Fiber optic    3198
DSL            2488
No             1581
Name: InternetService, dtype: int64
----------------
Tipo da coluna:
object
----------------


Com as informações sobre a contratação ou não do serviço de internet, também há uma diferenciação entre o tipo de fornecimento com `Fiber optic` e `DSL`.






#### Coluna 'OnlineSecurity'

In [ ]:
analisa_coluna('OnlineSecurity')

Valores únicos:
['No' 'Yes' 'No internet service']
----------------
Contagem de valores:
No                     3608
Yes                    2078
No internet service    1581
Name: OnlineSecurity, dtype: int64
----------------
Tipo da coluna:
object
----------------


De acordo com o dicionário, a coluna contém informações sobre a contratação ou não de um serviço adicional de segurança online. Os valores de `No internet service` correspondem a quantidade de `No` na coluna `InternetService`.

#### Coluna 'OnlineBackup'

In [ ]:
analisa_coluna('OnlineBackup')

Valores únicos:
['Yes' 'No' 'No internet service']
----------------
Contagem de valores:
No                     3182
Yes                    2504
No internet service    1581
Name: OnlineBackup, dtype: int64
----------------
Tipo da coluna:
object
----------------


Assim como a coluna anterior, há valores para `Yes`, `No` e também um terceiro valor, correspondendo aos clientes que não contrataram internet.

#### Coluna 'DeviceProtection'

In [ ]:
analisa_coluna('DeviceProtection')

Valores únicos:
['No' 'Yes' 'No internet service']
----------------
Contagem de valores:
No                     3195
Yes                    2491
No internet service    1581
Name: DeviceProtection, dtype: int64
----------------
Tipo da coluna:
object
----------------


Assim como a coluna anterior, há valores para `Yes`, `No` e também um terceiro valor, correspondendo aos clientes que não contrataram internet.

#### Coluna 'TechSupport'

In [ ]:
analisa_coluna('TechSupport')

Valores únicos:
['Yes' 'No' 'No internet service']
----------------
Contagem de valores:
No                     3582
Yes                    2104
No internet service    1581
Name: TechSupport, dtype: int64
----------------
Tipo da coluna:
object
----------------


Assim como a coluna anterior, há valores para `Yes`, `No` e também um terceiro valor, correspondendo aos clientes que não contrataram internet.

#### Coluna 'StreamingTV'

In [ ]:
analisa_coluna('StreamingTV')

Valores únicos:
['Yes' 'No' 'No internet service']
----------------
Contagem de valores:
No                     2896
Yes                    2790
No internet service    1581
Name: StreamingTV, dtype: int64
----------------
Tipo da coluna:
object
----------------


Assim como a coluna anterior, há valores para `Yes`, `No` e também um terceiro valor, correspondendo aos clientes que não contrataram internet.

#### Coluna 'StreamingMovies'

In [ ]:
analisa_coluna('StreamingMovies')

Valores únicos:
['No' 'Yes' 'No internet service']
----------------
Contagem de valores:
No                     2870
Yes                    2816
No internet service    1581
Name: StreamingMovies, dtype: int64
----------------
Tipo da coluna:
object
----------------


Assim como a coluna anterior, há valores para `Yes`, `No` e também um terceiro valor, correspondendo aos clientes que não contrataram internet.

#### Coluna 'Contract'

In [ ]:
analisa_coluna('Contract')

Valores únicos:
['One year' 'Month-to-month' 'Two year']
----------------
Contagem de valores:
Month-to-month    4005
Two year          1743
One year          1519
Name: Contract, dtype: int64
----------------
Tipo da coluna:
object
----------------


Há, na base de dados, 3 tipos de contrato disponíveis para os clientes. A maioria dos clientes optou por um contrato mês a mês.

#### Coluna 'PaperlessBilling'

In [ ]:
analisa_coluna('PaperlessBilling')

Valores únicos:
['Yes' 'No']
----------------
Contagem de valores:
Yes    4311
No     2956
Name: PaperlessBilling, dtype: int64
----------------
Tipo da coluna:
object
----------------


A maioria dos clientes prefere receber a fatura via e-mail.

#### Coluna 'PaymentMethod'

In [ ]:
analisa_coluna('PaymentMethod')

Valores únicos:
['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
----------------
Contagem de valores:
Electronic check             2445
Mailed check                 1665
Bank transfer (automatic)    1589
Credit card (automatic)      1568
Name: PaymentMethod, dtype: int64
----------------
Tipo da coluna:
object
----------------


Essa coluna contém informações sobre quais os métodos de pagamento. Há uma preferência pelo meio `Electronic check`. Sistemas automáticos de cobrança, seja por `Bank transfer` ou `Credit card` são também muito utilizados.

#### Coluna 'Charges.Monthly'

In [ ]:
analisa_coluna('Charges.Monthly')

Valores únicos:
[65.6  59.9  73.9  ... 91.75 68.8  67.85]
----------------
Contagem de valores:
20.05     65
19.85     46
19.90     46
19.70     45
19.55     45
          ..
23.45      1
116.55     1
106.85     1
68.55      1
67.85      1
Name: Charges.Monthly, Length: 1585, dtype: int64
----------------
Tipo da coluna:
float64
----------------


Segundo o dicionário, a coluna traz informações de valor mensal pago pelos clientes. Os valores possuem no máximo 2 casas decimais.

O tipo da coluna `float64` é o padrão que a biblioteca Pandas [utiliza para valores decimais](https://beepscore.com/website/2018/10/12/using-pandas-with-python-decimal.html). Variáveis do tipo `float` ou `float64` podem apresentar uma falta de acurácia, apesar de, na maioria dos casos, não haver problemas de análises. Caso essa falta de acurácia seja perceptível em análises futuras, será necessário fazer a alteração de tipo.

#### Coluna 'Charges.Total'

In [ ]:
analisa_coluna('Charges.Total')

Valores únicos:
['593.3' '542.4' '280.85' ... '742.9' '4627.65' '3707.6']
----------------
Contagem de valores:
           11
20.2       11
19.75       9
19.55       9
19.9        9
           ..
272         1
1426.45     1
371.6       1
6786.4      1
3707.6      1
Name: Charges.Total, Length: 6531, dtype: int64
----------------
Tipo da coluna:
object
----------------


Segundo o dicionário, a coluna traz informações de valor total pago por cada cliente. No entanto, a coluna está com o tipo errado: em vez de `object`, que significa que os dados estão sendo considerados como texto pelo sistema, o correto seria `float64`, como a coluna `Charges.Monthly`



### Considerações e pontos importantes

1. Será necessário tratar dados em branco da coluna 'Churn'. Como há clientes com 0 e 1 meses de contrato, será necessária uma análise mais clara dos dados, ajudando assim em uma decisão de se manter ou descartar os dados apresentados.

2. A coluna 'SeniorCitizen' possue valores '0' e '1', ao invés de 'Yes' e 'No' presente nas outras colunas booleanas. Isso será unificado para depois poder ser feita uma tradução dos dados.

3. As colunas extraídas durante o processo de normalização da base de dados aparentam ter informações condizentes entre si. 

4. Pode ser necessário alterar o tipo dos dados da coluna 'Charges.Monthly' para decimal.

5. O tipo da coluna 'Charges.Total' deve ser alterado para float64.

Com as análises feitas e com considerações a serem tomadas, é possível passar para a próxima etapa do processo, que seria a limpeza e transformação dos dados.

## Limpeza e preparação dos dados

Além das considerações e observações já feitas, estarei utilizando um [guia da plataforma Tableau](https://www.tableau.com/learn/articles/what-is-data-cleaning) que define o processo de limpeza dos dados, além de listar alguns passos que devem ser usados durante essa etapa.

### Conferindo Duplicatas

Durante as análises inicais, a base de dados não aparentava ter linhas duplicadas.

Para conferir e validar essa informação, podemos utilizar métodos da biblioteca Pandas. O [método `duplicated()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) retorna valores `False`, caso não haja valores duplicados, ou `True` para casos em que o valor esteja duplicado.

Primeiramente, a coluna `customerID` deve ser um código de cliente único, portanto não pode aceitar valores duplicados.

In [ ]:
dados_normal.duplicated(subset='customerID', keep='first').value_counts()

False    7267
dtype: int64

Como há apenas valores `False`, podemos afirmar que não há valores duplicados nesta coluna. 

Como há uma coluna sem valores duplicados, não há linhas duplicadas na base de dados. No entanto podemos refazer o passo e conferir para todo o dataset.

In [ ]:
dados_normal.duplicated(keep='first').value_counts()

False    7267
dtype: int64

Não havendo linhas duplicadas, podemos ir para o próximo passo sugerido no blog e efetuar a correção de erros estruturais do sistema.

### Inconsistências nas colunas

Segundo o blog do Tableau, erros estruturais ocorrem quando os dados não seguem um padrão, seja havendo erros de digitação, valores diferentes que passam a mesma informação ou falta de um padrão nos dados. 

Com essa definição e as observações feitas após a análise das colunas, há alguns erros estruturais.

#### 1. Coluna 'SeniorCitizen'

A coluna `SeniorCitizen` é uma coluna que não segue o padrão. Ao invés de utilizar 'Yes' ou 'No' para os valores, utiliza '1' e '0'.

Para isso, será usado o método [`.replace()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html).

In [ ]:
mapa = {1:'Yes', 0:'No'}

In [ ]:
dados_normal['SeniorCitizen'].replace(mapa, inplace=True)

In [ ]:
analisa_coluna('SeniorCitizen')

Valores únicos:
['No' 'Yes']
----------------
Contagem de valores:
No     6085
Yes    1182
Name: SeniorCitizen, dtype: int64
----------------
Tipo da coluna:
object
----------------


Com essa alteração, a coluna segue os padrões dos dados das outas colunas. Isso será importante quando os dados forem traduzidos.

#### 2. Coluna 'Charges.Total'

Essa coluna está com o tipo errado de valores. Ela representa o valor total pago pelo cliente, porém os dados estão sendo considerados como dados de texto. 

O tipo será o mesmo da coluna `Charges.Monthly`, que é `float64`.

Essa mudança será feita utilizando [`.to_numeric()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html). Esse método permite fazer a troca de tipo de maneira rápida. O próprio método define se o tipo será `float64` ou `int64`, dependendo dos valores na coluna.

In [ ]:
pd.to_numeric(dados_normal['Charges.Total'])

ValueError: ignored

Ocorreu um erro. Existe um valor em branco na coluna que não foi percebido nas análises iniciais. Esse valor deve ser tratado para 

In [ ]:
dados_normal.loc[dados_normal['Charges.Total'] == ' ']

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
975,1371-DWPAZ,No,Female,No,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,
1775,2520-SGTTA,No,Female,No,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,
1955,2775-SEFEE,No,Male,No,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,
2075,2923-ARZLG,No,Male,No,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,
2232,3115-CZMZD,No,Male,No,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,
2308,3213-VVOLG,No,Male,No,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,
2930,4075-WKNIU,No,Female,No,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,
3134,4367-NUYAO,No,Male,No,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,
3203,4472-LVYGI,No,Female,No,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,
4169,5709-LVOEQ,No,Female,No,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,


Os valores em branco na coluna correspondem aos clientes que possuem 0 meses de contrato. Esses cluentes ainda não efetuaram o pagamento da sua fatura. Por conta disso, irei trocar os valores que estão vazios para 0. Assim será possível manter os dados e as análises de maneira coerente.

In [ ]:
mapa = {' ': 0}

dados_normal['Charges.Total'].replace(mapa, inplace=True)

In [ ]:
pd.to_numeric(dados_normal['Charges.Total'])

0        593.30
1        542.40
2        280.85
3       1237.85
4        267.40
         ...   
7262     742.90
7263    1873.70
7264      92.75
7265    4627.65
7266    3707.60
Name: Charges.Total, Length: 7267, dtype: float64

Com a correção dos valores faltantes, foi possível fazer a conversão do tipo. O método utilizou o tipo `float64`, que será mantido.

### Explorando valores Faltantes

Além das inconsistências, também há valores faltantes. Algguns já foram tratados na coluna `Charges.Total`, porém também há valores faltantes na coluna `Churn`. 

Como o objetivo da empresa é encontrar clientes que possam querer cancelar, essa coluna será utilizada como meta no momento da criação de modelos de machine learning. Por isso ela precisa estar com valores precisos e em conformidade.

Primeiro, deve-se analisar mais profundamente os valores faltantes e tentar entender o motivo de eles existirem.

In [ ]:
faltantes = dados_normal.loc[dados_normal['Churn'] == '']

In [ ]:
faltantes

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
30,0047-ZHDTW,,Female,No,No,No,11,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),79.00,929.3
75,0120-YZLQA,,Male,No,No,No,71,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),19.90,1355.1
96,0154-QYHJU,,Male,No,No,No,29,Yes,No,DSL,...,Yes,No,Yes,No,No,One year,Yes,Electronic check,58.75,1696.2
98,0162-RZGMZ,,Female,Yes,No,No,5,Yes,No,DSL,...,Yes,No,Yes,No,No,Month-to-month,No,Credit card (automatic),59.90,287.85
175,0274-VVQOQ,,Male,Yes,Yes,No,65,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,One year,Yes,Bank transfer (automatic),103.15,6792.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,9840-GSRFX,,Female,No,No,No,14,Yes,Yes,DSL,...,Yes,No,No,No,No,One year,Yes,Mailed check,54.25,773.2
7180,9872-RZQQB,,Female,No,Yes,No,49,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,No,Bank transfer (automatic),40.65,2070.75
7211,9920-GNDMB,,Male,No,No,No,9,Yes,Yes,Fiber optic,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.25,684.85
7239,9955-RVWSC,,Female,No,Yes,Yes,67,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),19.25,1372.9


Analisando o dataset, não há uma variável que explique a falta de informações na coluna `Churn`.

De acordo com este [site](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/), essa é uma característica de dados MCAR, ou *Missing Completely At Random* (Faltando em Completa Aleatoriedade em tradução livre). Neste caso, é possível excluir os dados do modelo, apesar de perder alguns dados importantes.

In [ ]:
dados_normal['Churn'].value_counts(normalize=True)*100

No     71.198569
Yes    25.719004
        3.082427
Name: Churn, dtype: float64

Cerca de 3% dos dados estão faltantes. Para análises futuras e treinamento do modelo de machine learning, esses dados serão excluidos.

Como a empresa deseja entender o comportamento das pessoas antes de ser confirmada a evasão, esses dados serão mantidos, pois podem ser utilizados em testes dos modelos de machine learning.

### Considerações

1. Não há linhas duplicadas no dataset. A colua `customerID` contém apenas valores únicos.

2. A coluna `SeniorCitizen` possua valores que não estavam de acordo com o padrão, então os valores foram substitupidos.

3. Para poder fazer a conversão de tipos de dados na coluna `Charges.Total`, foram necessário tratar dados em branco que não foram percebidos anteriormente. Por conta das características dos dados, os valores faltantes foram substituidos por 0.

4. Os valores faltantes da coluna `Churn` serão separados dos dados finais. Não há informações suficientes para substituir os valores faltantes como 'Yes' ou 'No', porém eles podem ser úteis no momento de testes dos modelos de machine learning. 

Com as correções feitas, é possível realizar algumas etapas requisitadas pela empresa: traduzir as informações para o português, além de incluir uma coluna com gastos diários por cliente.

## Pedidos extras

A empresa fez dois pedidos extras: efetuar a tradução das colunas e criar uma coluna com gastos diários por cliente.

Serão traduzidos todos os dados e nomes de colunas que estão na língua inglesa. Além disso, o nome das colunas também será traduzido e seguirá uma nomenclatura padrão, coisa que não existe no momento.

### Traduções

#### Nome das colunas

Para realizar a tradução do nome das colunas, será utilizado o método [`.rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html) do Pandas.

In [ ]:
dados_normal.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [ ]:
colunas = {'customerID': 'ID_Cliente', 
           'Churn' : 'Evasao', 
           'gender': 'Genero', 
           'SeniorCitizen' : 'Eh_Idoso', 
           'Partner' : 'Tem_Parceiro',
           'Dependents' : 'Tem_Dependentes', 
           'tenure' : 'Tempo_Contrato', 
           'PhoneService' : 'Servico_Telefone', 
           'MultipleLines' : 'Linhas_Multiplas',
           'InternetService' : 'Servico_Internet', 
           'OnlineSecurity' : 'Adiconal_Seguranca', 
           'OnlineBackup' : 'Adicional_Backup', 
           'DeviceProtection' : 'Adiconal_Protecao',
           'TechSupport' : 'Adicional_Suporte', 
           'StreamingTV': 'Streaming_TV', 
           'StreamingMovies': 'Streaming_Filmes', 
           'Contract': 'Tipo_Contrato',
           'PaperlessBilling': 'Conta_Digital', 
           'PaymentMethod': 'Metodo_Pagamento', 
           'Charges.Monthly': 'Valor_Mensal',
           'Charges.Total': 'Valor_Total'
           }

In [ ]:
dados_normal.rename(mapper = colunas, axis = 1, inplace = True)

#### Dados

Para a tradução das colunas, será usado o método [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

In [ ]:
traducoes = {
    'Yes': 'Sim',
    'No': 'Não',
    'Male': 'Masculino',
    'Female': 'Feminino',
    'No phone service': 'Sem serviço de telefone',
    'No internet service': 'Sem serviço de internet',
    'Month-to-month' : 'Mensal',
    'Two year': 'Bianual',
    'One year': 'Anual',
    'Fiber optic': 'Fibra Óptica',
    'Bank transfer (automatic)': 'Tranferência bancária (automática)',
    'Credit card (automatic)': 'Cartão de crédito (automática)',
    'Electronic check': 'Cheque eletrônico',
    'Mailed check': 'Cheque',
}

Além de traduções diretas, algumas traduções foram simplificadas.

Os métodos de pagamento incluidos não sao usuais no Brasil, o que fez a tradução ser um pouco mais complicada. O sistema de pagamentos por meio de cheque eletrônico existe no país, o que permitiu a tradução. Já o hábito de mandar cheques por correios, incluindo para pagamento de contas, não é usual. Para esse, foi deixado apenas como 'Cheque'.

In [ ]:
dados_normal.replace(traducoes, inplace=True)

In [ ]:
dados_normal.head()

,ID_Cliente,Evasao,Genero,Eh_Idoso,Tem_Parceiro,Tem_Dependentes,Tempo_Contrato,Servico_Telefone,Linhas_Multiplas,Servico_Internet,...,Adicional_Backup,Adiconal_Protecao,Adicional_Suporte,Streaming_TV,Streaming_Filmes,Tipo_Contrato,Conta_Digital,Metodo_Pagamento,Valor_Mensal,Valor_Total
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,...,Sim,Não,Sim,Sim,Não,Anual,Sim,Cheque,65.6,593.3
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,...,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,59.9,542.4
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,...,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,73.9,280.85
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,...,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,98.0,1237.85
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,...,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,83.9,267.4


Para confirmar que os dados não sofreram nemhum erro, a função declarada no momento de análise de colunas será usada novamente, dessa vez dentro de um loop.

In [ ]:
for coluna in dados_normal.columns:
    print(coluna)
    analisa_coluna(coluna)

ID_Cliente
Valores únicos:
['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
----------------
Contagem de valores:
0002-ORFBO    1
6614-VBEGU    1
6637-KYRCV    1
6635-MYYYZ    1
6635-CPNUN    1
             ..
3374-TTZTK    1
3374-PZLXD    1
3374-LXDEV    1
3373-YZZYM    1
9995-HOTOH    1
Name: ID_Cliente, Length: 7267, dtype: int64
----------------
Tipo da coluna:
object
----------------
Evasao
Valores únicos:
['Não' 'Sim' '']
----------------
Contagem de valores:
Não    5174
Sim    1869
        224
Name: Evasao, dtype: int64
----------------
Tipo da coluna:
object
----------------
Genero
Valores únicos:
['Feminino' 'Masculino']
----------------
Contagem de valores:
Masculino    3675
Feminino     3592
Name: Genero, dtype: int64
----------------
Tipo da coluna:
object
----------------
Eh_Idoso
Valores únicos:
['Não' 'Sim']
----------------
Contagem de valores:
Não    6085
Sim    1182
Name: Eh_Idoso, dtype: int64
----------------
Tipo da coluna:
objec

Conferindo os valores únicos, percebe-se que a tradução foi bem sucedida. Pode-se então ser criada a coluna extra com dados de gastos por dia por cliente.

### Coluna `Valor_Dia`

Para a criação da coluna, será utilizada a coluna `Valor_Mensal`.

Em média um mês contém 30 dias e 9 horas. Por isso, o valor da coluna `Valor_Mensal` será dividido por 30. Como padrão, o valor será arredondado para que ele fique com apenas 2 casas decimais, assim como os outros valores financeiros contidos nos dados.

Em um primeiro momento, a coluna nova foi incluida no final do DataFrame. Apesar de funcionar, a empresa requisitou que esta coluna fosse incluida na posição 18. Por isso, alterei um pouco o código para ser possível utilizar o método [`.insert()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.insert.html).

Primeiramente, será criada uma série do Pandas com os valores, para que depois seja usada como argumento do método de inserção de coluna.

In [ ]:
valor_dia = round(dados_normal['Valor_Mensal']/30, 2)

In [ ]:
dados_normal.insert(17,'Valor_Dia', valor_dia)

In [ ]:
dados_normal.head()

,ID_Cliente,Evasao,Genero,Eh_Idoso,Tem_Parceiro,Tem_Dependentes,Tempo_Contrato,Servico_Telefone,Linhas_Multiplas,Servico_Internet,...,Adiconal_Protecao,Adicional_Suporte,Streaming_TV,Streaming_Filmes,Tipo_Contrato,Valor_Dia,Conta_Digital,Metodo_Pagamento,Valor_Mensal,Valor_Total
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,...,Não,Sim,Sim,Não,Anual,2.19,Sim,Cheque,65.6,593.3
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,...,Não,Não,Não,Sim,Mensal,2.00,Não,Cheque,59.9,542.4
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,...,Sim,Não,Não,Não,Mensal,2.46,Sim,Cheque eletrônico,73.9,280.85
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,...,Sim,Não,Sim,Sim,Mensal,3.27,Sim,Cheque eletrônico,98.0,1237.85
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,...,Não,Sim,Sim,Não,Mensal,2.80,Sim,Cheque,83.9,267.4


## Exportando os dados

Com os dados preparados e as requisições extras resolvidas, os dados podem ser exportados e salvos localmente. Essa exportação vai permitir análises futuras, seja em python ou em outros sistemas.

Como definido, os dados serão divididos de acordo com a coluna `Evasao`. Os valores em branco da coluna serão armazenados em um arquivo diferente, para que outras análises possam ser feitas.

### Dividindo os DataFrames

In [ ]:
dados_evasao_vazios = dados_normal.loc[dados_normal['Evasao'] == '']
dados_evasao_completos = dados_normal.loc[dados_normal['Evasao'] != '']

Para conferir, será utilizado o método [`.unique()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html)

In [ ]:
dados_evasao_vazios['Evasao'].unique()

array([''], dtype=object)

In [ ]:
dados_evasao_completos['Evasao'].unique()

array(['Não', 'Sim'], dtype=object)

### Exportando os dados

Os dados serão exportados para um arquivo do tipo csv.

In [ ]:
dados_evasao_vazios.to_csv('dados_evasao_vazio.csv', index = False)
dados_evasao_completos.to_csv('dados_evasao_completos.csv', index = False)